# Veritus AI Detector V2: High-Resolution Training (Fine-Tuned)

This notebook trains a **V2 Model** using **EfficientNetB0** (better than MobileNetV2).
It uses **Data Augmentation** and **Fine-Tuning** to achieve maximum accuracy on 224x224 images.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import pathlib
import shutil
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetB0

# 1. Download High-Res Dataset
print("Downloading High-Res Dataset...")
path = kagglehub.dataset_download("cashbowman/ai-generated-images-vs-real-images")
data_dir = pathlib.Path(path)
print("Path:", data_dir)

# --- FIX DATASET STRUCTURE ---
# The dataset has nested folders (e.g. AiArtData/AiArtData/image.jpg)
# We need to move files up one level.
for class_name in ['AiArtData', 'RealArt']:
    class_dir = data_dir / class_name
    nested_dir = class_dir / class_name
    
    if nested_dir.exists() and nested_dir.is_dir():
        print(f"Fixing nested structure for {class_name}...")
        # Move files up
        for file in nested_dir.iterdir():
            shutil.move(str(file), str(class_dir))
        # Remove empty nested folder
        nested_dir.rmdir()
        print(f"Fixed: {class_name} structure is now correct.")
    else:
        print(f"Structure for {class_name} is already correct.")

In [ ]:
# 2. Setup Data (224x224 Resolution)
BATCH_SIZE = 32
IMG_SIZE = (224, 224) # Standard for EfficientNet

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

class_names = train_ds.class_names
print("Classes:", class_names)

# Prefetching for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# 3. Build Model with Data Augmentation & EfficientNetB0

data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])

# Using EfficientNetB0 (Better accuracy than MobileNetV2)
base_model = EfficientNetB0(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False # Freeze base model initially

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
# EfficientNet expects 0-255 inputs
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1)(x)

model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
# 4. Stage 1: Train Head (Top Layers)
initial_epochs = 5
print("--- Stage 1: Training Head ---")
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

In [ ]:
# 5. Stage 2: Fine-Tuning (Unfreeze Base Model)
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Compile the model with a lower learning rate
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

model.summary()

fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

print("--- Stage 2: Fine-Tuning ---")
history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

In [ ]:
# 6. Save Best v2 Model
model.save('ai_detector_v2.h5')
print("High-Accuracy V2 Model saved as ai_detector_v2.h5")

In [ ]:
# 7. Plot Accuracy
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()